In [ ]:
from google.colab import drive
drive.mount('/content/drive')

root = '/content/drive/My Drive/surely_not_sarcasm_data/'

Mounted at /content/drive


In [ ]:
!pip install contractions
!cp -r '$root/utils' .

In [ ]:
import pandas as pd
import numpy as np
import pickle

from utils.preprocessing import preprocess_documents

# general preprocessing

In [ ]:
names = pd.read_csv(root+'data/key.csv', sep='\t').columns.tolist()
df = pd.read_csv(root+'data/train-balanced.csv', sep='\t', header=None, names=names)
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,1476662123,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,1477959850,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,1474580737,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,1476824627,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,1483117213,Yep can confirm I saw the tool they use for th...


In [ ]:
df_test = pd.read_csv(root+'data/test-balanced.csv', sep='\t', header=None, names=names)
df_test.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,Actually most of her supporters and sane peopl...,Quinnjester,politics,3,3,0,2016-09,1473569605,Hillary's Surrogotes Told to Blame Media for '...
1,0,They can't survive without an echo chamber whi...,TheGettysburgAddress,The_Donald,13,-1,-1,2016-11,1478788413,Thank God Liberals like to live in concentrate...
2,0,you're pretty cute yourself 1729 total,Sempiternally_free,2007scape,8,-1,-1,2016-11,1478042903,Saw this cutie training his Attack today...
3,0,If you kill me you'll crash the meme market,Catacomb82,AskReddit,2,-1,-1,2016-10,1477412597,If you were locked in a room with 49 other peo...
4,0,I bet he wrote that last message as he was sob...,Dorian-throwaway,niceguys,5,-1,-1,2016-11,1477962278,You're not even that pretty!


In [ ]:
df.shape, df_test.shape

((1010826, 10), (251608, 10))

In [ ]:
df_all = pd.concat([df, df_test], ignore_index=True)
df_all.shape

(1262434, 10)

In [ ]:
def preprocess(df_org):
    print(fr'missing values: {df_org.isna().sum()}')
    print('')
    df = df_org.dropna()
    duplicates = df[['label', 'comment', 'author', 'subreddit', 'date', 'parent_comment']].duplicated()
    print(fr'{df[duplicates].shape[0]} duplicates')
    df = df[~duplicates].copy()
    df.drop(['author', 'subreddit', 'score', 'ups', 'downs', 'date', 'created_utc'], axis=1, inplace=True)

    df = df[~df['comment'].str.lower().str.endswith(' the')]
    df = df[~df['comment'].str.lower().str.endswith('forgot your')]
    df = df[~df['comment'].str.lower().str.endswith('dropped your')]
    df = df[~df['comment'].str.lower().str.endswith('this:')]

    df['comment'] = (df['comment']
         .str.replace(r'&amp;nbsp;', ' ', regex=False)
         .str.replace(r'&amp;', '&', regex=False)
         .str.replace(r'&lt;', '<', regex=False)
         .str.replace(r'&gt;', '>', regex=False)
         .str.replace(r'&quot;', '"', regex=False)
    )

    df['parent_comment'] = (df['parent_comment']
            .str.replace(r'&amp;nbsp;', ' ', regex=False)
            .str.replace(r'&amp;', '&', regex=False)
            .str.replace(r'&lt;', '<', regex=False)
            .str.replace(r'&gt;', '>', regex=False)
            .str.replace(r'&quot;', '"', regex=False)
    )

    return df

In [ ]:
df_all = preprocess(df_all)

missing values: label              0
comment           67
author             0
subreddit          0
score              0
ups                0
downs              0
date               0
created_utc        0
parent_comment     0
dtype: int64

383 duplicates


In [ ]:
df_all.to_csv(root+'data/all_balanced_partly_preprocessed.csv', index=False)

# preprocess text

In [ ]:
data = {}
data['raw_comment'] = df_all['comment']
data['raw_parent'] = df_all['parent_comment']
data['label'] = df_all['label']
data['comment'] = preprocess_documents(df_all['comment'], root=root)
data['parent'] = preprocess_documents(df_all['parent_comment'], root=root)

with open(root+'all_preprocessed_data_2105', 'wb') as file:
    pickle.dump(data, file)